## C S 329E HW 9

# Crossvalidation and hyperparameter selection

## Mark Borjas (and your partner's name if you are working in a pair)

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [2]:
df = load_diabetes()
df_X = df['data']
s_y = df['target']

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [3]:
def get_linear_regression_model( df_X, s_y ):
    [beta_hat, residuals, rank, s] = np.linalg.lstsq(df_X, s_y, rcond=-1)
    return beta_hat

In [4]:
np.random.seed(23)
test_X = pd.DataFrame(np.random.random((34, 4)))
one_array = np.ones((len(test_X), 1))
test_X = np.hstack((test_X, one_array))
beta_hat = get_linear_regression_model(test_X, pd.Series(np.random.random(34) * 10.0))
beta_hat

array([ 1.77890808,  0.74032569, -1.3506416 ,  0.14535984,  4.18818425])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [5]:
def partition_data( df_X, s_y, k ):
    dict_k_df_X = {}
    dict_k_s_y = {}
    cur_idx = 0
    
    for i in range(1, k):
        partition_size = int(np.round(len(df_X) / k + (np.random.normal() * len(df_X) /(k * 20))))
        dict_k_df_X[i] = df_X[cur_idx:cur_idx + partition_size]
        dict_k_s_y[i] = s_y[cur_idx:cur_idx + partition_size]
        cur_idx += partition_size
        one_array = np.ones((len(dict_k_df_X[i]), 1))
        dict_k_df_X[i] = np.hstack((dict_k_df_X[i], one_array))
    
    dict_k_df_X[k] = df_X[cur_idx:len(df_X)]
    dict_k_s_y[k] = s_y[cur_idx:len(df_X)]
    
    one_array = np.ones((len(dict_k_df_X[k]), 1))
    dict_k_df_X[k] = np.hstack((dict_k_df_X[k], one_array))
    
    return (dict_k_df_X, dict_k_s_y)

In [6]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [7]:
sum_folds = 0

for key, values in dict_k_df_X.items():
    sum_folds += len(values)
    print("Fold " + str(key) + " length of dataframe is " + str(len(values)) + " and length of series is " + str(len(dict_k_s_y[key])))
    
print("The sum of the number of elements in each fold is " + str(sum_folds) + " and there are " + str(len(df_X)) + " rows in the original df")

Fold 1 length of dataframe is 91 and length of series is 91
Fold 2 length of dataframe is 78 and length of series is 78
Fold 3 length of dataframe is 91 and length of series is 91
Fold 4 length of dataframe is 85 and length of series is 85
Fold 5 length of dataframe is 97 and length of series is 97
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [8]:
def get_mae( s_y, s_y_hat):
    mae = 0 # mean absolute error
    for i in range(len(s_y)):
        mae += abs(s_y[i] - s_y_hat[i]) / len(s_y)
    return mae

In [9]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [12]:
mae = np.array([])
for i in dict_k_df_X.keys():
    first_found = False
    # Build linear regression model from training set
    # i is the testing set
    
    # for loop iterates to build the training set. 
    # if i = j, skips it because it's the testing set

    for j in dict_k_df_X.keys():
        if (i != j):
            if (first_found == False):
                training_df_X = dict_k_df_X[j]
                training_s_y = dict_k_s_y[j]
                first_found = True
            else:
                training_df_X = np.append(training_df_X, dict_k_df_X[j], axis=0)
                training_s_y = np.append(training_s_y, dict_k_s_y[j], axis=0)
            
    beta_hat = np.reshape(get_linear_regression_model(training_df_X, training_s_y), (11, 1))
    s_y_hat = np.matmul(dict_k_df_X[i], beta_hat)
    mae = np.append(mae, get_mae(dict_k_s_y[i], s_y_hat))

In [13]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))


The min MAE is 40.49, the max MAE is 47.57, and the mean MAE is 44.62


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [14]:
df = load_iris()
df
df_X = df['data']
s_y = df['target']

# shuffle
newOrder = np.arange(len(s_y))
np.random.shuffle(newOrder)

d = df_X.copy()
s= s_y.copy()

##################### create new array with newOrder 

for i in range(len(newOrder)):
    df_X[i] = d[newOrder[i]]
    s_y[i] = s[newOrder[i]]

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [15]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [16]:
def get_acc( s_1, s_2 ):
    if len(s_1) != len(s_2):
        return "Lengths do not equal each other"
    n = len(s_1)
    c = 0.0;
    for i in range(n):
        if s_1[i] == s_2[i]:
            c += 1
    return c/n;

In [17]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [18]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    print("Execution #" + str(k))
    test_df_X = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
    first_found = False

    for j in dict_k_df_X.keys():
        if (k != j):
            if (first_found == False):
                outerFold_df_X = dict_k_df_X[j]
                outerFold_s_y = dict_k_s_y[j]
                first_found = True
            else:
                outerFold_df_X = np.append(outerFold_df_X, dict_k_df_X[j], axis=0)
                outerFold_s_y = np.append(outerFold_s_y, dict_k_s_y[j], axis=0)
    
    (dict_Inner_df_X, dict_Inner_s_y) = partition_data(outerFold_df_X, outerFold_s_y, 4)
    this_acc = np.array([])
    for pos_min_impurity in possible_min_impurity_decrease:
        inner_acc = np.array([]);
        for m in dict_Inner_df_X.keys():
            inner_test_df_X = dict_Inner_df_X[m]
            inner_test_s_y = dict_Inner_s_y[m]
            first_found = False
            for n in dict_Inner_df_X.keys():
                if (m!=n):
                    if (first_found== False):
                        innerFold_df_X = dict_Inner_df_X[n]
                        innerFold_s_y = dict_Inner_s_y[n]
                    else:
                        innerFold_df_X = np.append(innerFold_df_X, dict_Inner_df_X[n], axis=0)
                        innerFold_s_y = np.append(innerFold_s_y, dict_Inner_s_y[n], axis=0)
            
            clf = tree.DecisionTreeClassifier(min_impurity_decrease=pos_min_impurity, criterion="gini")
            clf = clf.fit(innerFold_df_X, innerFold_s_y)
            innerPrediction = clf.predict(inner_test_df_X)
            inner_acc = np.append(inner_acc, get_acc(innerPrediction, inner_test_s_y))
            #print(tree.export_text(clf))
        print("Testing " + str(pos_min_impurity) + " min impurity decrease")
        print("\tAverage accuracy over 4 folds is " + str(np.round(inner_acc.mean(), 2)))
        this_acc = np.append(this_acc, inner_acc.mean())
        
        
    # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
    print("Best min impurity " + str(possible_min_impurity_decrease[np.argmax(this_acc)]))
    print("")      
    # Use best min impurity decrease to train model
    clf = tree.DecisionTreeClassifier(min_impurity_decrease=possible_min_impurity_decrease[np.argmax(this_acc)], criterion="gini")
    clf = clf.fit(outerFold_df_X, outerFold_s_y)
    outerPrediction = clf.predict(test_df_X)
    
    acc = get_acc(outerPrediction,test_s_y)
    # outer accuracy calculation 
    outer_acc = np.append(outer_acc, acc) # make sure and calculate this_acc in your loop
    print("Outer accuracy " + str(np.round(acc * 100, 2)) + " %\n")

Execution #1
Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.91
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.7
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.61
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.29
Best min impurity 0.1

Outer accuracy 96.88 %

Execution #2
Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.88
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.67
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.59
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.26
Best min impurity 0.1

Outer accuracy 90.32 %

Execution #3
Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.71
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.71
Testing 0.4 min impurity decrease
	Average accuracy over 4 fol

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [19]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(outer_acc.min(),outer_acc.max(),outer_acc.mean()))

The min MAE is 0.88, the max MAE is 1.00, and the mean MAE is 0.95
